In [1]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
from define import qcode as qc
from define.submission import Submission
from define.load import LoadSub
from define.fnames import PhysicalChannel
from define.chanreps import ConvertRepresentations

In [2]:
def ProcessToChi(process):
    chi = np.zeros_like(process, dtype = np.complex128)
    q = 0
    for p in process:
        chi[q] = ConvertRepresentations(p, "process", "chi")
        q+=1
    return chi
# def ProcessToChi(process):
#     return np.array(list(map(lambda P: ConvertRepresentations(P, "process", "chi"), process)), dtype=np.complex128)

In [3]:
def Kron(*mats):
    """
    Kronecker product of a list of matrices.
    """
    if len(mats) <= 1:
        return mats[0]
    return np.kron(mats[0], Kron(*mats[1:]))

In [4]:
def GetPauliPhase(qcode):
    nstabs = 2**(qcode.N - qcode.K)
    ops_LST = qcode.PauliOperatorsLST
    lookup = qcode.lookup.astype(int)
    phases = np.ones(len(ops_LST), dtype = np.complex128)
    prods = np.zeros(((len(ops_LST),qcode.N)),dtype=np.int)
    for op_index in range(len(ops_LST)):
        t_op = np.mod(op_index, nstabs)
        # print("t_op : {} m1 : {} m2: {}".format(t_op, lookup[:,2:][t_op], ops_LST[op_index]))
        (prods[op_index], phases[op_index]) = qc.PauliProduct(lookup[:,2:][t_op], ops_LST[op_index])
    return prods, phases

In [5]:
def LogicalInfidelity(chi, qcode, phases):
    nstabs = 2**(qcode.N - qcode.K)
    log_fid = 0
    lexicographic = np.sum(np.power(4, np.arange(qcode.N))[::-1] * qcode.PauliOperatorsLST, axis=1)
    for t in range(nstabs):
        for j in range(t*nstabs,(t+1)*nstabs):
            indj = qcode.PauliCorrectableIndices[j]
            lindj = lexicographic[j]
            for k in range(t*nstabs,(t+1)*nstabs):
                indk = qcode.PauliCorrectableIndices[k]
                lindk = lexicographic[k]
                term = chi[lindj,lindk] * phases[indj] * np.conj(phases[indk])
                log_fid += term
    return (1 - np.real(log_fid))

In [6]:
def ImpactOfRC(nqchi, nqchi_RC, qcode, phases):
    # Logical infidelity for the non RC channel.
    loginfid_noRC = LogicalInfidelity(nqchi, qcode, phases)
    # Logical infidelity with RC.
    loginfid_RC = LogicalInfidelity(nqchi_RC, qcode, phases)
    impact = loginfid_noRC/loginfid_RC
    return (loginfid_noRC, loginfid_RC, impact)

In [9]:
# Load the elements of the code.
qcode = qc.QuantumErrorCorrectingCode("Steane")
qc.Load(qcode)
nstabs = 2**(qcode.N - qcode.K)
(prods, phases) = GetPauliPhase(qcode)

In [10]:
# Load the elements of the (non RC and RC) channels.
submit = Submission()
exists = LoadSub(submit, "degradation", 1, 0)
process = np.reshape(np.load(PhysicalChannel(submit, submit.noiserates[0, :])), [7, 4, 4])
chi = ProcessToChi(process)
nqchi = Kron(*chi)
nqchi_RC = np.diag(np.diag(nqchi))

In [11]:
LogicalInfidelity(nqchi, qcode, phases)

0.00025853811738152377

In [12]:
LogicalInfidelity(nqchi_RC, qcode, phases)

-6.661338147750939e-16

In [ ]:
(perf_RC, perf_nonRC, impact) = ImpactOfRC(nqchi, nqchi_RC, qcode, phases)
print("non RC performance: {}\nRC performance: {}\nImpact: nonRC/RC = {}.".format(perf_nonRC, perf_RC, impact))

In [ ]:
def PLog(mat):
    """
    Take a log where non-posvitive entries are set to 1 before applying the log function, so that they can be set to zero in the result.
    """
    cutoff = 1E-3
    mat = np.abs(mat)
    mat[mat <= cutoff] = cutoff
    return np.log10(mat)

In [ ]:
plt.matshow(PLog(chi_tensored), cmap="gray")
plt.colorbar()
plt.savefig("matrix_plot.png", format="png", dpi=300)

In [ ]:
gain = 0
degradation = 0
Repeat several times:
    1. Take a random correctable error E
    2. E' = E.S
    3. Look at the chi-matrix element x = chi_{E,E'}
    4. If x is positive, then it adds to degradation; else it adds to a gain.
                                               
effect = gain - degradation